In [ ]:
import pandas, os, re, sys, matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
fname = "/Users/dfporter/pma/clip/experiments/exp27/exp27_linker_linker_dimer_exps.xlsx"
df = pandas.read_excel(fname, sheetname='gel2 dots combined')

In [ ]:
def melt(_df):
    #red, green = zip(_df.IR700, _df.IR800)
    #(lane, l5, l3) = zip(_df.Lane, _df.L5, _df.L3)
    to_dict = []
    for n, (red, green, lane, l5, l3, obj, recovery) in enumerate(zip(
            _df.IR700, _df.IR800, _df.Lane, _df.L5, _df.L3, _df.Object, _df['Recovery IR800'].tolist())):
        to_dict.append({'Lane': lane, 'L5': l5, 'L3': l3, 'Fluorescence': red/10000, 'Channel': '700',
                       'Object': obj, 'Recovery': recovery})
        to_dict.append({'Lane': lane, 'L5': l5, 'L3': l3, 'Fluorescence': green/10000, 'Channel': '800',
                       'Object': obj, 'Recovery': recovery})
    return pandas.DataFrame(to_dict)
df = pandas.read_excel(fname, sheetname='gel2 dots combined')

df = df[df['Object']!='Input per IP'].copy()
df['Object'] = [re.sub('\AInput$', 'Gel extract (TBE)', x) for x in df.Object]
#df = df[df['Object']!='Input'].copy() # lambda df: df.A > 0, :
def lower_if_input(x, y):
    if x == 'Gel extract (TBE)':
        return y / 5
    else:
        return y

def order(_df):
    def get_num(label):
        if label == 'TBE-Urea gel':
            return 0
        elif label == 'Gel extract (TBE)':
            return 1
        elif label == 'DNAse elution':
            return 10
        else:
            return 2
    _df['Order'] = [get_num(x) for x in _df.Object]
    _df.sort_index(by='Order', inplace=True)
    return(_df)
df = order(df)
print(df)
m = melt(df)
m['Fluorescence'] = [lower_if_input(obj, val) for (obj, val) in zip(m.Object, m.Fluorescence)]

m = m[m['Object']!='Gel extract (TBE)']

m = order(m)
print(m)
def figs(m, df, show=True):
    fig, ax = plt.subplots(ncols=2, nrows=1)
    sns.barplot(x='Fluorescence', y='Object', hue='Channel', data=m, ax=ax[0])
    ax[0].set_xlabel('Fluorescence (10,000s)')

    df = df[df['Object']!='DNAse elution']
    sns.barplot(x='L5/L3', y='Object', data=df, ax=ax[1])
    ax[1].set_xlabel('Ratio of 700/800 channel fluorescence')
    
    plt.tight_layout()
    fig.set_figheight(3)
    fig.set_figwidth(10)
    if show:
        plt.show()
    else:
        plt.savefig('/Users/dfporter/pma/figs/dimer_IPs_lanes_combined.pdf')

    plt.clf()
    plt.close()
figs(m, df)
figs(m, df, show=False)

In [ ]:
#print(df)
def melt(_df):
    #red, green = zip(_df.IR700, _df.IR800)
    #(lane, l5, l3) = zip(_df.Lane, _df.L5, _df.L3)
    to_dict = []
    for n, (red, green, lane, l5, l3) in enumerate(zip(_df.IR700, _df.IR800, _df.Lane, _df.L5, _df.L3)):
        print(n, red, green)
        to_dict.append({'Lane': lane, 'L5': l5, 'L3': l3, 'Fluorescence': red/10000, 'Channel': '700'})
        to_dict.append({'Lane': lane, 'L5': l5, 'L3': l3, 'Fluorescence': green/10000, 'Channel': '800'})

    return pandas.DataFrame(to_dict)

inputs = df[df['Object']=='Input'].copy()
inputs = melt(inputs)
gel = melt(df[df['Object']=='TBE-Urea gel'].copy())
water = melt(df[df['Object']=='Water elution'].copy())
formamide = melt(df[df['Object']=='Formamide elution'].copy())
DNAse = melt(df[df['Object']=='DNAse elution'].copy())

fig, ax = plt.subplots(ncols=2, nrows=3, sharex=False)
sns.barplot(ax=ax[0][0], y='Fluorescence', x='Lane', data=gel, hue='Channel', 
            palette={'800': 'lightgreen', '700': 'tomato'})
sns.barplot(ax=ax[0][1], y='Fluorescence', x='Lane', data=inputs, hue='Channel', 
            palette={'800': 'lightgreen', '700': 'tomato'})
sns.barplot(ax=ax[1][0], y='Fluorescence', x='Lane', data=water, hue='Channel', 
            palette={'800': 'lightgreen', '700': 'tomato'})
sns.barplot(ax=ax[1][1], y='Fluorescence', x='Lane', data=formamide, hue='Channel', 
            palette={'800': 'lightgreen', '700': 'tomato'})
sns.barplot(ax=ax[2][0], y='Fluorescence', x='Lane', data=DNAse, hue='Channel', 
            palette={'800': 'lightgreen', '700': 'tomato'})
fig.set_figwidth(val=6)
fig.set_figheight(val=8)
ax[0][0].set_title('Gel')
ax[0][1].set_title('Gel extract')
ax[1][0].set_title('Water+biotin eluate')
ax[1][1].set_title('Formamide eluate')
ax[2][0].set_title('DNAse eluate')

plt.tight_layout()
print(inputs)
plt.show()

In [ ]:
fname = "/Users/dfporter/pma/clip/experiments/exp24 hnRNPC and linker only/exp24.xlsx"
df = pandas.read_excel(fname, sheetname='flat linker only inputs')
def rep_cols(df):
    if type(df) == type([]):
        cols = df
    else:
        cols = df.columns
    return [x for x in cols if re.search('[Rr][Ee][Pp]', x)]
print(rep_cols(df))
df['Sample'] = [' input, '.join([x, y]) for x, y in zip(df['Input'].tolist(), df.IP)]
df['Sample'] = [re.sub('IP', 'bait', x) for x in df.Sample]
df['Sample'] = [re.sub('None input, Buffer', 'Water', x) for x in df.Sample]
xlabel = 'Fluorescence Units (thousands)'
df[xlabel] = [x/1000 for x in df['Fluorescence Units'].tolist()]

fig = plt.figure()
sns.stripplot(x=xlabel, y='Sample', hue='Channel', data=df, orient='horizontal', 
            palette={800: 'lightgreen', 700: 'salmon'})
fig.savefig('../figs/linker_purification_specificity_stripplot.pdf')
plt.show()
plt.close()

fig = plt.figure()
sns.barplot(x='Fluorescence Units', y='Sample', hue='Channel', data=df, orient='horizontal', 
            palette={800: 'lightgreen', 700: 'crimson'})
plt.show()
plt.close()

